#Calculation of additional merices

In this notebook following metrics are calculated for textss generated with SFT-n and SFT-nRL-n models:

1.  SLOR
2.  Readability score (FRE)
3.  Coherence Score






## Download and import libraries

In [ ]:
!pip install transformers trl nltk textstat

In [ ]:
!python -m spacy download de_core_news_sm

2023-12-09 15:37:45.017414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 15:37:45.017468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 15:37:45.017502: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 15:37:46.383196: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 30.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelWithLMHead,GPT2Model,TrainingArguments, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from google.colab import drive
import pandas as pd
import random
import textstat
import numpy as np
import torch as nn
import math
import torch
import torch.nn.functional as F
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import de_core_news_sm
import gensim.corpora as corpora
import gensim
from gensim.models import CoherenceModel
import string
import torch.nn.functional as F
import spacy
from collections import Counter
stop_words = stopwords.words('german')

drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prepare Evaluation
### Set up model and tokenizer

In [ ]:
tokenizer_ref = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
model_ref = AutoModelWithLMHead.from_pretrained("dbmdz/bert-base-german-cased")
tokenizer_ref.pad_token = tokenizer_ref.eos_token
global tokenizer_ref
global model_ref

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Create function for calculation of SLOR.

This is based on the paper "Sentence-Level Fluency Evaluation: References Help, But Can Be Spared!" by Kann et al. 2020. (https://arxiv.org/pdf/1809.08731.pdf)


In [ ]:
nlp = spacy.load("de_core_news_sm")

def slor_score(text):
  text=text.lstrip()
  slor_scores=[]
  doc = nlp(text)

  # Extract sentences
  sentences = [sent.text for sent in doc.sents]
  # Calculate word frequencies
  for sentence in sentences:

    tokenized_input = tokenizer_ref.encode(sentence, return_tensors="pt",truncation=True)
    sentence_length = len(tokenized_input[0])

    loss, logits = model_ref(tokenized_input, labels=tokenized_input)[:2]
    softmax = torch.nn.Softmax(dim=-1)
    probabilities = softmax(logits)[0]

  # Calculate sentence probability by multiplying token probabilities
    sentence_probability = 1.0
    for token_probability in probabilities:
      max_token_probability = token_probability.max().item()
      sentence_probability *= max_token_probability

    words = re.findall(r'\b\w+\b', sentence.lower())
    word_frequencies = Counter(words)
    total_words = len(words)

    # Calculate probabilities for each word
    word_probabilities = {word: count / total_words for word, count in word_frequencies.items()}

    # Display probabilities for each word
    unigram=1
    for word, prob in word_probabilities.items():
        unigram=unigram*prob
    if total_words!=0:
      slor=math.log(sentence_probability)/total_words-math.log(unigram)/total_words
      slor_scores.append(slor)

  return np.mean(slor_scores)

### Calculate Coherence Score

Based on https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [ ]:
def lda_coherence_score(text):
    stop_words = set(stopwords.words('german'))
    data_words = nltk.word_tokenize(text, language='german')
    data_words = [word for word in data_words if word.lower() not in stop_words]

    bigram = gensim.models.Phrases([data_words], min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    data_words_bigrams = [bigram_mod[data_words]]

    nlp = spacy.load("de_core_news_sm", disable=['parser', 'ner'])
    data_lemmatized = [
        [token.lemma_ for token in nlp(word)] for word in data_words_bigrams[0]
    ]

    id2word = corpora.Dictionary(data_lemmatized)
    corpus = [id2word.doc2bow(text) for text in data_lemmatized]

    lda_model = gensim.models.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=4,
                                            random_state=100,
                                            chunksize=100,
                                            passes=20,
                                            per_word_topics=True)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    return coherence_lda

In [ ]:
def lda_coherence_score(text):
    stop_words = set(stopwords.words('german'))
    data_words = nltk.word_tokenize(text, language='german')
    data_words = [word for word in data_words if word.lower() not in stop_words]

    bigram = gensim.models.Phrases([data_words], min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    data_words_bigrams = [bigram_mod[data_words]]

    nlp = spacy.load("de_core_news_sm", disable=['parser', 'ner'])
    data_lemmatized = [
        [token.lemma_ for token in nlp(word)] for word in data_words_bigrams[0]
    ]

    id2word = corpora.Dictionary(data_lemmatized)
    corpus = [id2word.doc2bow(text) for text in data_lemmatized]

    lda_model = gensim.models.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=4,
                                            random_state=100,
                                            chunksize=100,
                                            passes=20,
                                            per_word_topics=True)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    return coherence_lda

## Evaluation

### GPT-2

In [ ]:
gpt=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_GPT_org_longer.tsv" , sep="\t")
gpt["slor"]=gpt["text"].apply(slor_score)
gpt["readability_score"]=gpt["text"].apply(textstat.flesch_reading_ease)
gpt['text_processed'] = gpt['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
gpt['lda_score']=gpt['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",gpt["slor"].mean(),"avg. readability: ",gpt["readability_score"].mean(),"avg. coherence score: ",gpt["lda_score"].mean())

avg. slor:  1.110557875050443 avg. readability:  75.76570000000001 avg. coherence score:  0.863449721533332


### SFT1

In [ ]:
sft1=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_df_sft_1_longer.tsv", sep="\t")
sft1["slor"]=sft1["text"].apply(slor_score)
sft1["readability_score"]=sft1["text"].apply(textstat.flesch_reading_ease)

sft1['text_processed'] = sft1['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
sft1['lda_score']=sft1['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",sft1["slor"].mean(),"avg. readability: ",sft1["readability_score"].mean(),"avg. coherence score: ",sft1["lda_score"].mean())

avg. slor:  1.046170042532423 avg. readability:  79.9064 avg. coherence score:  0.8630368705696904


### SFT2

In [ ]:
sft2=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_df_sft_2_longer.tsv", sep="\t")
sft2["slor"]=sft2["text"].apply(slor_score)
sft2["readability_score"]=sft2["text"].apply(textstat.flesch_reading_ease)

sft2['text_processed'] = sft2['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
sft2['lda_score']=sft2['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",sft2["slor"].mean(),"avg. readability: ",sft2["readability_score"].mean(),"avg. coherence score: ",sft2["lda_score"].mean())

avg. slor:  1.0796542108706249 avg. readability:  79.8778 avg. coherence score:  0.8641129772973595


### SFT3

In [ ]:
sft3=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_df_sft_3_longer.tsv", sep="\t")
sft3["slor"]=sft3["text"].apply(slor_score)
sft3["readability_score"]=sft3["text"].apply(textstat.flesch_reading_ease)

sft3['text_processed'] = sft3['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
sft3['lda_score']=sft3['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",sft3["slor"].mean(),"avg. readability: ",sft3["readability_score"].mean(),"avg. coherence score: ",sft3["lda_score"].mean())

avg. slor:  1.0958362196293967 avg. readability:  78.6079 avg. coherence score:  0.8635499540599835


### SFT1RL1

In [ ]:
sft1_rl1=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_df_sft_1_rl_1_longer.tsv" , sep="\t")
sft1_rl1["slor"]=sft1_rl1["text"].apply(slor_score)
sft1_rl1["readability_score"]=sft1_rl1["text"].apply(textstat.flesch_reading_ease)

sft1_rl1['text_processed'] = sft1_rl1['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
sft1_rl1['lda_score']=sft1_rl1['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",sft1_rl1["slor"].mean(),"avg. readability: ",sft1_rl1["readability_score"].mean(),"avg. coherence score: ",sft1_rl1["lda_score"].mean())

avg. slor:  0.9645056365104776 avg. readability:  79.4986 avg. coherence score:  0.8639004312892162


### SFT1RL2

In [ ]:
sft1_rl2=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_df_sft_1_rl_2_longer.tsv" , sep="\t")
sft1_rl2["slor"]=sft1_rl2["text"].apply(slor_score)
sft1_rl2["readability_score"]=sft1_rl2["text"].apply(textstat.flesch_reading_ease)

sft1_rl2['text_processed'] = sft1_rl2['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
sft1_rl2['lda_score']=sft1_rl2['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",sft1_rl2["slor"].mean(),"avg. readability: ",sft1_rl2["readability_score"].mean(),"avg. coherence score: ",sft1_rl2["lda_score"].mean())

avg. slor:  0.9946805911058757 avg. readability:  81.4764 avg. coherence score:  0.8660099248267025


### SFT2RL1

In [ ]:
sft2_rl1=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_df_sft_2_rl_1_longer.tsv" , sep="\t")
sft2_rl1["slor"]=sft2_rl1["text"].apply(slor_score)
sft2_rl1["readability_score"]=sft2_rl1["text"].apply(textstat.flesch_reading_ease)

sft2_rl1['text_processed'] = sft2_rl1['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
sft2_rl1['lda_score']=sft2_rl1['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",sft2_rl1["slor"].mean(),"avg. readability: ",sft2_rl1["readability_score"].mean(),"avg. coherence score: ",sft2_rl1["lda_score"].mean())

avg. slor:  0.9646878708872572 avg. readability:  75.3697 avg. coherence score:  0.8647585782637435


### SFT1RL2

In [ ]:
sft2_rl2=pd.read_csv("/content/drive/MyDrive/Masterthesis/Results/Evaluation_df_sft_2_rl_2_longer.tsv" , sep="\t")
sft2_rl2["slor"]=sft2_rl2["text"].apply(slor_score)
sft2_rl2["readability_score"]=sft2_rl2["text"].apply(textstat.flesch_reading_ease)

sft2_rl2['text_processed'] = sft2_rl2['text'].map(lambda x: x.lower()).map(lambda x: re.sub('[,\.!?]', '', x))
sft2_rl2['lda_score']=sft2_rl2['text_processed'].apply(lda_coherence_score)

In [ ]:
print("avg. slor: ",sft2_rl2["slor"].mean(),"avg. readability: ",sft2_rl2["readability_score"].mean(),"avg. coherence score: ",sft2_rl2["lda_score"].mean())

avg. slor:  0.8693090729379628 avg. readability:  79.307 avg. coherence score:  0.8639540989732357


topk

In [ ]:
df_top_k11=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_k11.tsv', sep="\t")
df_top_k11["slor"]=df_top_k11["gen_text"].apply(slor_score)
df_top_k11["readability_score"]=df_top_k11["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_k11["slor"].mean(),"avg. readability: ",df_top_k11["readability_score"].mean())

In [ ]:
df_top_k12=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_k12.tsv', sep="\t")
df_top_k12["slor"]=df_top_k12["gen_text"].apply(slor_score)
df_top_k12["readability_score"]=df_top_k12["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_k12["slor"].mean(),"avg. readability: ",df_top_k12["readability_score"].mean())

In [ ]:
df_top_k21=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_k21.tsv', sep="\t")
df_top_k21["slor"]=df_top_k21["gen_text"].apply(slor_score)
df_top_k21["readability_score"]=df_top_k21["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_k21["slor"].mean(),"avg. readability: ",df_top_k21["readability_score"].mean())

In [ ]:
df_top_k22=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_k21.tsv', sep="\t")
df_top_k22["slor"]=df_top_k22["gen_text"].apply(slor_score)
df_top_k22["readability_score"]=df_top_k22["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_k22["slor"].mean(),"avg. readability: ",df_top_k22["readability_score"].mean())

topp

In [ ]:
df_top_p11=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_p_11.tsv', sep="\t")
df_top_p11["slor"]=df_top_p11["gen_text"].apply(slor_score)
df_top_p11["readability_score"]=df_top_p11["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_p11["slor"].mean(),"avg. readability: ",df_top_p11["readability_score"].mean())

In [ ]:
df_top_p12=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_p_12.tsv', sep="\t")
df_top_p12["slor"]=df_top_p12["gen_text"].apply(slor_score)
df_top_p12["readability_score"]=df_top_p12["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_p12["slor"].mean(),"avg. readability: ",df_top_p12["readability_score"].mean())

In [ ]:
df_top_p21=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_p_21.tsv', sep="\t")
df_top_p21["slor"]=df_top_p21["gen_text"].apply(slor_score)
df_top_p21["readability_score"]=df_top_p21["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_p21["slor"].mean(),"avg. readability: ",df_top_p21["readability_score"].mean())

In [ ]:
df_top_p22=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_p_22.tsv', sep="\t")
df_top_p22["slor"]=df_top_p22["gen_text"].apply(slor_score)
df_top_p22["readability_score"]=df_top_p22["gen_text"].apply(textstat.flesch_reading_ease)

In [ ]:
print("avg. slor: ",df_top_p22["slor"].mean(),"avg. readability: ",df_top_p22["readability_score"].mean())

avg. slor:  1.0380814427505753 avg. readability:  83.3466
